In [1]:
import pandas as pd
import numpy as np

from average_conn_V3 import _mask_brain, _simulate_img,_extract_time_series_voxel, _extract_time_series_parcels, _average_conn, _load_voxel_data, _resample_mist, _compute_tseries, _build_size_roi, _compute_afc, _compute_brute_afc 

IndentationError: expected an indented block after function definition on line 48 (average_conn_V3.py, line 51)

# Load data

In [ ]:
from nilearn.datasets import fetch_icbm152_brain_gm_mask, fetch_development_fmri, fetch_atlas_basc_multiscale_2015
import nibabel as nib

# Load mask of parcels
parcellation = fetch_atlas_basc_multiscale_2015(version='sym')['scale064']
img_mist = nib.load(parcellation)

# Grey matter segmentation from the MNI template 
mask_mni = fetch_icbm152_brain_gm_mask()

# Fetch demo dataset
data = fetch_development_fmri(n_subjects=1, reduce_confounds=True)
imgs = data.func[0:10]
confounds = data.confounds[0:10]

# test private functions

In [ ]:
mask_brain = _mask_brain(imgs, mask_mni)
parcel_brain, parcel_voxel = _parcel_brain(img_mist, mask_brain)

## It works on simple data

# Now for whole-brain intra-network average connectivity

We first get the mask of parcels, demo dataset, and build a group combined mask

In [ ]:
from nilearn.image import math_img
img_mist, imgs, confounds, mask_combined = _load_parcels_compute_mask()


#

Optionally visualise the mask

In [ ]:
#from nilearn.plotting import plot_roi
#plot_roi(mask_combined, resampling_interpolation='nearest')

Load fMRI data for one subject

In [ ]:
img_n, time_series_voxel = _load_voxel_data(mask_combined, imgs[0], confounds[0])

Compute the time series using mist resampled

In [ ]:
time_series_mist, masker_mist = _compute_tseries(mist_resampled, mask_combined, img_n)

Compute the AFC and also get the parcel sizes

In [ ]:
afc, size_parcels = _compute_afc(time_series_mist, masker_mist, mist_voxel)

## Compute AFC by brute force for all parcels

In [ ]:
brute_afc = _compute_brute_afc(time_series_voxel, mist_voxel, masker_mist)

Compare the two methods

In [ ]:
num_parcel = 50
print(f"parcel number {num_parcel}, size {size_parcels[num_parcel - 1]}: 'NIAK' AFC ={afc[num_parcel - 1]}, 'brute force' AFC ={brute_afc[num_parcel - 1]}")

## So, they don't match. Suggestion from Hao-Ting: it might be the mask difference - test has a perfect overlapping mask and parcel, and real data does not (mask_combined, mist_resampled)